In [3]:
from typing import List, Protocol
from dataclasses import dataclass

# Observer Protocol
class YieldCurveObserver(Protocol):
    def on_curve_update(self, curve: 'YieldCurve') -> None:
        ...

@dataclass
class SwapDefinition:
    notional: float
    fixed_rate: float
    maturity_years: int
    # ... other static swap terms

class YieldCurve:
    def __init__(self, rates: dict):
        self.rates = rates
        self._observers: List[YieldCurveObserver] = []
    
    def subscribe(self, observer: YieldCurveObserver):
        self._observers.append(observer)
    
    def update_rates(self, new_rates: dict):
        self.rates = new_rates
        self._notify_observers()
    
    def _notify_observers(self):
        for observer in self._observers:
            observer.on_curve_update(self)

class Swap:
    def __init__(self, definition: SwapDefinition, curve: YieldCurve):
        self.definition = definition
        self.curve = curve
        self.pv = 0.0
        
        # Subscribe to curve updates
        curve.subscribe(self)
        self._calculate_pv()
    
    def on_curve_update(self, curve: YieldCurve):
        """Called automatically when curve changes"""
        self._calculate_pv()
    
    def _calculate_pv(self):
        # Your PV calculation logic here
        self.pv = self._compute_present_value()
    
    def _compute_present_value(self) -> float:
        # Simplified example
        discount_rate = self.curve.rates.get(self.definition.maturity_years, 0.05)
        return self.definition.notional / ((1 + discount_rate) ** self.definition.maturity_years)

# Usage
curve = YieldCurve(rates={1: 0.02, 5: 0.03, 10: 0.04})

swaps = [
    Swap(SwapDefinition(1, 0.03, 5), curve),
    Swap(SwapDefinition(2, 0.035, 10), curve),
]

print(f"Before: {[s.pv for s in swaps]}")

# When curve updates, all swaps automatically recalculate
curve.update_rates({1: 0.025, 5: 0.035, 10: 0.045})

print(f"After: {[s.pv for s in swaps]}")

Before: [0.8626087843841639, 1.3511283376515972]
After: [0.8419731668585242, 1.287855364060086]
